In [ ]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "<insert subscription key here>"
endpoint = "<insert your service endpoint>"
img_url = "<url to image here>"
def getText(subscription_key, endpoint, img_url):
  computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
  '''
  END - Authenticate
  '''

  '''
  OCR: Read File using the Read API, extract text - remote
  This example will extract text in an image, then print results, line by line.
  This API call can also extract handwriting style text (not shown).
  '''
  print("===== Read File - remote =====")
  # Get an image with text
  read_image_url = img_url

  # Call API with URL and raw response (allows you to get the operation location)
  read_response = computervision_client.read(read_image_url,  raw=True)

  # Get the operation location (URL with an ID at the end) from the response
  read_operation_location = read_response.headers["Operation-Location"]
  # Grab the ID from the URL
  operation_id = read_operation_location.split("/")[-1]

  # Call the "GET" API and wait for it to retrieve the results 
  while True:
      read_result = computervision_client.get_read_result(operation_id)
      if read_result.status not in ['notStarted', 'running']:
          break
      time.sleep(1)

  # Print the detected text, line by line
  texts = []
  if read_result.status == OperationStatusCodes.succeeded:
      for text_result in read_result.analyze_result.read_results:
          for line in text_result.lines:
              texts.append(line.text)

  return texts

In [ ]:
texts = getText(subscription_key, endpoint, img_url)

===== Read File - remote =====


In [ ]:
texts

['DEPARTMENT OF STATE',
 'CEPTIS',
 'A',
 'FOREIGN SERVICE OF THE UNITED STATES OF AMERICA',
 'ANNUIT',
 'Certification of Birth Abrnad',
 'of a Citizen of the United States of America',
 'This is to certify that according to records on file in this Office',
 'Sex',
 'MALE',
 'was born at',
 'US NAVAL HOSPITAL, OKINAWA, JAPAN',
 '071',
 'AUGUST 13, 1990',
 'Report of birth recorded on',
 'SEPTEMBER 14, 1990',
 'In Witness Whereof, I have hereunto subscribed my name and affixed the seat of the Consular Service of the United States',
 'of America at',
 'NAHA, JAPAN',
 'this',
 '14TH',
 'day of',
 '_SEPTEMBER',
 '19.90',
 '(SEAL)',
 'CONSUL _ of the United States of America',
 'WARNING: This certificate is not valid if it has been altered in any way whatsoever or if it does not bear the raised seal of the',
 'office of Issuance.',
 'Form FS-545',
 '10-75597-2',
 'GPO',
 '1-73']

In [ ]:
#checks if text extracted belong to a social security card
def isSocialSecurity(texts):
  for text in texts:
    if "SOCIAL SECURITY" in text:
      return True
  return False

In [ ]:
isSocialSecurity(texts)

True

In [ ]:
def getSSCDetails(texts):
  info = {}
  info['Document Type'] = 'Social Security Card'
  ssn_probable = [text for text in texts if \
              all([len(text) == 11,
                  text[:3].isdigit(),
                  text[3] == '-',
                  text[4:6].isdigit(),
                  text[6] == '-',
                  text[-4:].isdigit()
              ])]
  
  if len(ssn_probable) == 1:
    info['ssn'] = ssn_probable[0]

  return info

In [ ]:
sscdetails = getSSCDetails(texts)

In [ ]:
sscdetails

{'ssn': '562-14-5542'}

In [ ]:
def isBirthCertificate(texts):
  for text in texts:
    if "birth certifiacte" in text  or "certification of birth" in text or "was born" in text:
      return True
  return False

In [ ]:
check_texts = []
def getBirthCDetails(texts):
  flag = False
  info = {}
  info['Document Type'] = 'Birth Certificate'
  months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
  years = [*range(1800,2023)]
  for text in texts:
    check_texts.append(text)
    if "MALE" in text.upper(): 
      info['sex'] = "MALE"

    if "FEMALE" in text.upper():
      info['sex'] = "FEMALE"

    for month in months:
      if month.upper() in text.upper():
        for year in years:
          if str(year) in text:
            for check_text in check_texts:
              if 'WAS BORN' in check_text.upper():
                flag = True
                info['DOB'] = text.upper()
      if flag:
        break
  
  return info


In [ ]:
def pipeline(img_url):
  texts = getText(subscription_key, endpoint, img_url)
  if isSocialSecurity(texts):
    info = getSSCDetails(texts)
    if info != {}:
      return info

  if isBirthCertificate(texts):
    info = getBirthCDetails(texts)
    if info != {}:
      return info

In [ ]:
info = pipeline(img_url)

===== Read File - remote =====


In [ ]:
info

{'Document Type': 'Birth Certificate', 'sex': 'MALE', 'DOB': 'AUGUST 13, 1990'}